# ANALYSING BOND FUTURES CONTRACTS

In this notebook I analyse a CME bond futures contract

In [1]:
import datetime as dt

In [32]:
import pandas as pd

In [2]:
from financepy.products.bonds import *
from financepy.utils import *

####################################################################
#  FINANCEPY BETA Version 0.370 - This build: 28 Oct 2024 at 20:26 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Example from CME

https://www.cmegroup.com/education/files/understanding-treasury-futures.pdf

I am looking at Table 3 on Page 20

In [17]:
first_delivery_dt = Date(1, 12, 2017)
last_delivery_dt = Date(28, 12, 2017)

contract_size = 100000
contractCoupon = 0.06

# December 2017 contract
bondFutureContract = BondFuture("TYZ7", first_delivery_dt, last_delivery_dt,
                                   contract_size, contractCoupon)

In [6]:
print(bondFutureContract)

OBJECT TYPE: BondFuture
TICKER NAME: TYZ7
FIRST DELIVERY DATE: 01-DEC-2017
LAST DELIVERY DATE: 28-DEC-2017
CONTRACT SIZE: 100000
COUPON: 0.06



In [7]:
freq = FrequencyTypes.SEMI_ANNUAL
basis = DayCountTypes.ACT_ACT_ICMA
settle_dt = Date(10, 10, 2017)
issue_dt = Date(1, 1, 2000) # Dummy

In [10]:
bonds = []
prices = []
bond = Bond(issue_dt, Date(15, 8, 2027), 0.0225, freq, basis); bonds.append(bond); prices.append(99 + 1 / 32)
bond = Bond(issue_dt, Date(15, 5, 2027), 0.02375, freq, basis); bonds.append(bond); prices.append(100 + 5 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 2, 2027), 0.0225, freq, basis); bonds.append(bond); prices.append(99 + 5 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 11, 2026), 0.02, freq, basis); bonds.append(bond); prices.append(97 + 7 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(5, 8, 2026), 0.015, freq, basis); bonds.append(bond); prices.append(93 + 14 / 32)
bond = Bond(issue_dt, Date(15, 5, 2026), 0.01625, freq, basis); bonds.append(bond); prices.append(94 + 21 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 2, 2026), 0.01625, freq, basis); bonds.append(bond); prices.append(94 + 29 / 32)
bond = Bond(issue_dt, Date(15, 11, 2025), 0.0225, freq, basis); bonds.append(bond); prices.append(99 + 25 / 32)
bond = Bond(issue_dt, Date(15, 8, 2025), 0.02, freq, basis); bonds.append(bond); prices.append(98 + 3 / 32)
bond = Bond(issue_dt, Date(15, 5, 2025), 0.02125, freq, basis); bonds.append(bond); prices.append(99 + 5 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 2, 2025), 0.02, freq, basis); bonds.append(bond); prices.append(98 + 14 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 11, 2024), 0.0225, freq, basis); bonds.append(bond); prices.append(100 + 9 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 8, 2024), 0.02375, freq, basis); bonds.append(bond); prices.append(101 + 7 / 32 + 1 / 64)
# bond = Bond(issue_dt, Date(15, 8, 2024), 0.01875, freq, basis); bonds.append(bond); prices.append(98 + 1 / 32 + 0/64) #TYPO IN REPT

Calculate the bond yield

   Coupon      Bond Maturity        Yield  Clean Price
  2.25000        15-AUG-2027    2.36049984     99.03125
  2.37500        15-MAY-2027    2.35463188    100.17188
  2.25000        15-FEB-2027    2.34885040     99.17188
  2.00000        15-NOV-2026    2.33900466     97.23438
  1.50000        05-AUG-2026    2.32695325     93.43750
  1.62500        15-MAY-2026    2.31166907     94.67188
  1.62500        15-FEB-2026    2.29879314     94.90625
  2.25000        15-NOV-2025    2.27946129     99.78125
  2.00000        15-AUG-2025    2.26615960     98.09375
  2.12500        15-MAY-2025    2.24388311     99.17188
  2.00000        15-FEB-2025    2.22903638     98.45312
  2.25000        15-NOV-2024    2.20429165    100.29688
  2.37500        15-AUG-2024    2.17956249    101.23438


In [37]:
# Get the Invoice Prices
futures_price = 125 + 8/32 + 1/64

Get the conversion factors

In [ ]:
data = [] 

for bond, clean_price in zip(bonds, prices):
    yld = bond.yield_to_maturity(settle_dt, clean_price)
    dt = bond.maturity_dt
    cf = bondFutureContract.conversion_factor(bond)
    pip = bondFutureContract.principal_invoice_price(bond, futures_price)
    gainloss, payForBond, receiveOnFuture = bondFutureContract.delivery_gain_loss(bond, clean_price, futures_price)

    data.append({
        "Coupon": bond.cpn * 100,
        "Maturity": dt,
        "Yield": yld * 100,
        "Clean Price": clean_price,
        "TCF": cf,
        "PIP": pip,
        "Receive on Future": receiveOnFuture,
    })

# Create the DataFrame
df = pd.DataFrame(data)

# Optional: Format the DataFrame for display
df = df.round({
    "Coupon (%)": 5,
    "Yield to Maturity (%)": 8,
    "Clean Price": 5,
    "Conversion Factor": 5,
    "PIP": 2
})

# Display the DataFrame
print(df)

    Coupon     Maturity     Yield  Clean Price      TCF          PIP
0    2.250  15-AUG-2027  2.360500     99.03125  73.1429   9162291.08
1    2.375  15-MAY-2027  2.354632    100.17188  74.5496   9338502.24
2    2.250  15-FEB-2027  2.348850     99.17188  74.2122   9296237.62
3    2.000  15-NOV-2026  2.339005     97.23438  73.0756   9153860.71
4    1.500  05-AUG-2026  2.326953     93.43750  70.3762   8815718.68
5    1.625  15-MAY-2026  2.311669     94.67188  71.8566   9001161.91
6    1.625  15-FEB-2026  2.298793     94.90625  72.5226   9084588.82
7    2.250  15-NOV-2025  2.279461     99.78125  77.0269   9648822.77
8    2.000  15-AUG-2025  2.266160     98.09375  76.1241   9535732.96
9    2.125  15-MAY-2025  2.243883     99.17188  77.4870   9706457.48
10   2.000  15-FEB-2025  2.229036     98.45312  77.4079   9696548.97
11   2.250  15-NOV-2024  2.204292    100.29688  79.4343   9950387.24
12   2.375  15-AUG-2024  2.179562    101.23438  80.7241  10111954.84


In [9]:
# Get the Invoice Prices
futures_price = 125 + 8/32 + 1/64

In [43]:
ctd = bondFutureContract.cheapest_to_deliver(bonds, prices, futures_price)
print("CTD MATURITY", "CTD COUPON")
print(str(ctd.maturity_dt), ctd.cpn*100)

CTD MATURITY CTD COUPON
15-AUG-2024 2.375


Copyright (c) 2019, Dominic O'Kane 